## 需求：

### 主要目标
idea：为函数的parameter生成注释，一个函数可能有多个parameter。
利用数据集（JAVA）训练baseline模型，为parameter生成注释，并计算BLEU，ROUGE，METEOR指标。

实验设计（暂定）：

1. 定义训练粒度为一个item，每个item为`<code, parameter, parameter comment>`。
2. 在train数据集上baseline训练模型（valid用于指导epoch更新）。在训练时，可以将code和parameter直接拼起来，以`<code+parameter, parameter comment>`作为一个pair进行训练。
3. 在test上生成，并计算出指标。

4. baseline（备选）：
DeepCom，Transformer，BiLSTM，或其他容易复现的模型，越方便越好。

5. 输出结果（表格）：

方法名 | BLEU(1-4) | ROUGE | METEOR
---- | ---- | ---- |---- |
XXX | XXX | XXX | XXX
XXX | XXX | XXX | XXX


## 2 数据集：(valid, test, train)

### 说明：
1. 数据来自CodeSearchNet，已经利用tree-sitter做了解析，抽取了函数的parameter和相应的parameter comments。

数据的粒度是函数，初步统计如下
valid：5183
test：10955
train：164923 

2. 数据格式为jsonl（一行一个json），简单的读取方式为pandas
pandas.readjson("file/path", lines=True)

3. 数据有四个文件夹，每个文件夹都有子文件夹/java代表语言。
raw：CodeSearchNet的原始数据。
signature：抽取出来的函数及其参数，例如`int main(int a, String b)`
param_doc：抽取出来的参数注释，例如`@param a comment for a`
param_generation：（主要关注这个数据）合并了signature和param_doc。

整理出来的目录结构如下：

```
ParameterDescriptionDataset
├── param_doc
│   └── java
│       ├── test.jsonl
│       ├── train.jsonl
│       └── valid.jsonl
├── param_generation
│   └── java
│       ├── test.jsonl
│       ├── train.jsonl
│       └── valid.jsonl
├── raw
│   └── java
│       ├── test.jsonl
│       ├── train.jsonl
│       └── valid.jsonl
└── signature
    └── java
        ├── test.jsonl
        ├── train.jsonl
        └── valid.jsonl
```

4. 数据集格式为一个函数及其多个参数，`<method_code, [param1, param2, ...], [param_doc1, param_doc2, ...]>`
参数注释param_doc可能会缺失或者漏掉，前期统计发现（@param的数量/总的函数中的param的数量），比例不到50%。

5. 需要将`<method_code, [param1, param2, ...], [param_doc1, param_doc2, ...]>` 数据首先处理成`<code, parameter, parameter comment>`，然后进一步进行训练。
比如说一个`method_code` => `int main(int a, string b){XXX}`可以处理出两个item： 

`<method, int a, @param a>`

`<method, int b, @param b>`

缺失了param comment的可以直接去掉。

6. 下面展示具体的例子:

在`param_generation`中，每个item对应数据集中的一个代码片段，有四个字段：

method_num: 函数数量，一般是1个

list：函数列表，只取第一个即可

doc_param_num：参数注释的数量

doc_param：参数注释



In [6]:
import pandas as pd
from IPython.display import display

path = "path/to/data"
valid_path = "/data/ncc_data/code_search_net_feng/param_generation/java/valid.jsonl"

valid_data = pd.read_json(valid_path, lines=True)
display(valid_data)

,method_num,list,doc_param_num,doc_param
0,1,"[{'method_index_in_code_snippet': 0, 'paramete...",0,[]
1,1,"[{'method_index_in_code_snippet': 0, 'paramete...",0,[]
2,1,"[{'method_index_in_code_snippet': 0, 'paramete...",0,[]
3,1,"[{'method_index_in_code_snippet': 0, 'paramete...",0,[]
4,1,"[{'method_index_in_code_snippet': 0, 'paramete...",0,[]
...,...,...,...,...
5178,1,"[{'method_index_in_code_snippet': 0, 'paramete...",0,[]
5179,1,"[{'method_index_in_code_snippet': 0, 'paramete...",2,"[[@param klass, The Configuration Class to lo..."
5180,1,"[{'method_index_in_code_snippet': 0, 'paramete...",0,[]
5181,1,"[{'method_index_in_code_snippet': 0, 'paramete...",0,[]


In [18]:
# 一个method的数据例子
display(valid_data["list"][233][0])
display(valid_data["doc_param"][233])

{'method_index_in_code_snippet': 0,
 'parameter number': 2,
 'method_name': 'elementAt',
 'parameter_list': ['int index', 'T defaultValue'],
 'string': 'public final Mono<T> elementAt(int index, T defaultValue) '}

[['@param index', 'zero-based index of the only item to emit'],
 ['@param defaultValue', 'a default value to emit if the sequence is shorter']]

### 函数数据的解释：

'parameter number': 参数数量

'parameter_list'：提取出来的参数列表

'string'：整个函数的string

## 3 最后结合数据阐述需求

例如上面的例子，一个函数有两个参数['int index', 'T defaultValue']，首先将其处理出两个item：

<code_string, param=`int index`, param_doc=`(@param index) zero-based index of the only item to emit`>

<code_string, param=`T defaultValue`, param_doc=`(@param defaultValue) a default value to emit if the sequence is shorter`>


以此类推，train/test/valid都能处理出若干这样的item。

然后根据code sumamrization的流行的范式，用train的item来训练模型（valid验证），在test上生成结果BLEU，ROUGE，METEOR等。

例如，用BLEU的脚本，计算<已有的参数注释，生成的参数注释>的指标。

